In [161]:
import numpy as np
import pandas as pd

In [162]:
dataset=pd.read_csv("dataset.csv")

In [163]:
dataset.head()

,Defect No.,Location,Defect Description,Breach,Remediation
0,Building B/\nUnit 67,"Bed 1, sliding door",Stopper restricting the opening of the balcony...,Home Building Act 1989\nFailure to perform wor...,Remove the existing door stopper.\nInstall in ...
1,Building B/\nUnit 67,"Balcony, handrail",Glass is not identifiable as safety glass.,AS 1288 is a document referenced by the BCA 20...,Remove top rail;\nRemove glass;\nInstall new g...
2,Building B/\nUnit 67,"Balcony, Setdown",The concrete slab has been placed without a se...,Performance Solution to be provided outlining ...,The alternate solution required for the design...
3,Building B/\nUnit 67,"Balcony 1, Air condenser",The mechanical AC condenser has not been fixed...,Daikin Installation Guide requires the followi...,Set up the work area\nMark fixing locations;\n...
4,Building B/\nUnit 67,"Balcony, Air condenser",Open penetration in the external wall.\n\nTher...,Building Code of Australia 2016\n\nJ1.2 Therma...,Degas and remove AC Unit.\nInstall expanding f...


In [164]:
dataset = dataset[~dataset.apply(lambda row: row.astype(str).str.contains('No access').any(), axis=1)]
dataset = dataset[~dataset.apply(lambda row: row.astype(str).str.contains('Defect Description').any(), axis=1)]

In [165]:
dataset['Remediation'] = dataset['Remediation'].apply(lambda x: str(x) if not isinstance(x, str) else x)

# Apply the operation to remove lines starting with 'See section'
dataset['Remediation'] = dataset['Remediation'].apply(lambda x: '\n'.join([line for line in str(x).split('\n') if not line.strip().startswith('See section')]))

In [166]:
dataset['Breach'] = dataset['Breach'].apply(lambda x: '\n'.join([line for line in x.split('\n') if not line.strip().startswith('See section')]))

In [167]:
dataset['Breach'] = dataset['Breach'].apply(lambda x: x.rstrip('\n') if isinstance(x, str) else x)

# Remove newline characters from the end of strings in the 'Remediation' column
dataset['Remediation'] = dataset['Remediation'].apply(lambda x: x.rstrip('\n') if isinstance(x, str) else x)

In [168]:
dataset = dataset[['Defect Description', 'Breach', 'Remediation']]

In [169]:
dataset.head()

,Defect Description,Breach,Remediation
0,Stopper restricting the opening of the balcony...,Home Building Act 1989\nFailure to perform wor...,Remove the existing door stopper.\nInstall in ...
1,Glass is not identifiable as safety glass.,AS 1288 is a document referenced by the BCA 20...,Remove top rail;\nRemove glass;\nInstall new g...
2,The concrete slab has been placed without a se...,Performance Solution to be provided outlining ...,The alternate solution required for the design...
3,The mechanical AC condenser has not been fixed...,Daikin Installation Guide requires the followi...,Set up the work area\nMark fixing locations;\n...
4,Open penetration in the external wall.\n\nTher...,Building Code of Australia 2016\n\nJ1.2 Therma...,Degas and remove AC Unit.\nInstall expanding f...


In [170]:
result = pd.DataFrame()

# Iterate over each row in the original DataFrame
for index, row in dataset.iterrows():
    # Append the original row
    result = pd.concat([result, row.to_frame().T], ignore_index=True)
    # Append a copy of the original row
    result = pd.concat([result, row.to_frame().T], ignore_index=True)

In [171]:
result.head()

,Defect Description,Breach,Remediation
0,Stopper restricting the opening of the balcony...,Home Building Act 1989\nFailure to perform wor...,Remove the existing door stopper.\nInstall in ...
1,Stopper restricting the opening of the balcony...,Home Building Act 1989\nFailure to perform wor...,Remove the existing door stopper.\nInstall in ...
2,Glass is not identifiable as safety glass.,AS 1288 is a document referenced by the BCA 20...,Remove top rail;\nRemove glass;\nInstall new g...
3,Glass is not identifiable as safety glass.,AS 1288 is a document referenced by the BCA 20...,Remove top rail;\nRemove glass;\nInstall new g...
4,The concrete slab has been placed without a se...,Performance Solution to be provided outlining ...,The alternate solution required for the design...


In [172]:

def determine_instruction(index):
    if index % 2 == 0:
        return 'Tell the breach for the given defect'
    else:
        return 'Tell the remedy for the given breach'

# Add the 'Instruction' column to the original DataFrame using apply function
result['Instruction'] = result.index.to_series().apply(lambda x: determine_instruction(x))

# Copy the original DataFrame to create 'new_result'
new_result = result.copy()

# Alternatively, if you want to drop the index and reset it in 'new_result'
# new_result = result.copy().reset_index(drop=True)

# Update the original DataFrame with the new DataFrame
result = new_result


In [173]:
new_result.head()

,Defect Description,Breach,Remediation,Instruction
0,Stopper restricting the opening of the balcony...,Home Building Act 1989\nFailure to perform wor...,Remove the existing door stopper.\nInstall in ...,Tell the breach for the given defect
1,Stopper restricting the opening of the balcony...,Home Building Act 1989\nFailure to perform wor...,Remove the existing door stopper.\nInstall in ...,Tell the remedy for the given breach
2,Glass is not identifiable as safety glass.,AS 1288 is a document referenced by the BCA 20...,Remove top rail;\nRemove glass;\nInstall new g...,Tell the breach for the given defect
3,Glass is not identifiable as safety glass.,AS 1288 is a document referenced by the BCA 20...,Remove top rail;\nRemove glass;\nInstall new g...,Tell the remedy for the given breach
4,The concrete slab has been placed without a se...,Performance Solution to be provided outlining ...,The alternate solution required for the design...,Tell the breach for the given defect


In [174]:
new_result['Input'] = new_result.apply(lambda row: row['Defect Description'] if row.name % 2 == 0 else row['Breach'], axis=1)

new_result.head()

,Defect Description,Breach,Remediation,Instruction,Input
0,Stopper restricting the opening of the balcony...,Home Building Act 1989\nFailure to perform wor...,Remove the existing door stopper.\nInstall in ...,Tell the breach for the given defect,Stopper restricting the opening of the balcony...
1,Stopper restricting the opening of the balcony...,Home Building Act 1989\nFailure to perform wor...,Remove the existing door stopper.\nInstall in ...,Tell the remedy for the given breach,Home Building Act 1989\nFailure to perform wor...
2,Glass is not identifiable as safety glass.,AS 1288 is a document referenced by the BCA 20...,Remove top rail;\nRemove glass;\nInstall new g...,Tell the breach for the given defect,Glass is not identifiable as safety glass.
3,Glass is not identifiable as safety glass.,AS 1288 is a document referenced by the BCA 20...,Remove top rail;\nRemove glass;\nInstall new g...,Tell the remedy for the given breach,AS 1288 is a document referenced by the BCA 20...
4,The concrete slab has been placed without a se...,Performance Solution to be provided outlining ...,The alternate solution required for the design...,Tell the breach for the given defect,The concrete slab has been placed without a se...


In [175]:
new_result['Output'] = new_result.apply(lambda row: row['Breach'] if row.name % 2 == 0 else row['Remediation'], axis=1)

new_result = new_result[['Instruction', 'Input', 'Output']]

new_result.head()

,Instruction,Input,Output
0,Tell the breach for the given defect,Stopper restricting the opening of the balcony...,Home Building Act 1989\nFailure to perform wor...
1,Tell the remedy for the given breach,Home Building Act 1989\nFailure to perform wor...,Remove the existing door stopper.\nInstall in ...
2,Tell the breach for the given defect,Glass is not identifiable as safety glass.,AS 1288 is a document referenced by the BCA 20...
3,Tell the remedy for the given breach,AS 1288 is a document referenced by the BCA 20...,Remove top rail;\nRemove glass;\nInstall new g...
4,Tell the breach for the given defect,The concrete slab has been placed without a se...,Performance Solution to be provided outlining ...


In [176]:
breach_phrases = [
    "Tell the breach for the given defect",
    "Identify the breach corresponding to the provided defect.",
    "Provide details of the breach related to the specified defect.",
    "Describe the breach associated with the given defect.",
    "Report the breach linked to the provided defect.",
    "Explain the breach corresponding to the given defect.",
    "Specify the breach for the provided defect.",
    "State the breach related to the specified defect.",
    "Outline the breach associated with the given defect.",
    "Communicate the breach corresponding to the provided defect.",
    "Enumerate the breach for the specified defect."
]

remedy_phrases = [
    "Tell the remedy for the given breach"
    "Provide the solution for the specified breach.",
    "Explain the remedy related to the provided breach.",
    "Offer the resolution for the given breach.",
    "Describe the corrective action for the specified breach.",
    "Propose the remedy corresponding to the provided breach.",
    "Communicate the solution for the given breach.",
    "Outline the remedy associated with the specified breach.",
    "Detail the corrective measure for the provided breach.",
    "Present the solution corresponding to the given breach.",
    "Suggest the remedy for the specified breach."
]

In [177]:
new_result['Instruction'] = new_result['Instruction'].apply(lambda x: np.random.choice(breach_phrases) if x.strip() == 'Tell the breach for the given defect' else x)
new_result['Instruction'] = new_result['Instruction'].apply(lambda x: np.random.choice(remedy_phrases) if x.strip() == 'Tell the remedy for the given breach' else x)

C:\Users\prath\AppData\Local\Temp\ipykernel_156\3564849768.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_result['Instruction'] = new_result['Instruction'].apply(lambda x: np.random.choice(breach_phrases) if x.strip() == 'Tell the breach for the given defect' else x)
C:\Users\prath\AppData\Local\Temp\ipykernel_156\3564849768.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_result['Instruction'] = new_result['Instruction'].apply(lambda x: np.random.choice(remedy_phrases) if x.strip() == 'Tell

In [178]:
new_result.head()

,Instruction,Input,Output
0,Tell the breach for the given defect,Stopper restricting the opening of the balcony...,Home Building Act 1989\nFailure to perform wor...
1,Present the solution corresponding to the give...,Home Building Act 1989\nFailure to perform wor...,Remove the existing door stopper.\nInstall in ...
2,Provide details of the breach related to the s...,Glass is not identifiable as safety glass.,AS 1288 is a document referenced by the BCA 20...
3,Tell the remedy for the given breachProvide th...,AS 1288 is a document referenced by the BCA 20...,Remove top rail;\nRemove glass;\nInstall new g...
4,Tell the breach for the given defect,The concrete slab has been placed without a se...,Performance Solution to be provided outlining ...


In [179]:
shuffled_result = new_result.sample(frac=1).reset_index(drop=True)

shuffled_result.head()

,Instruction,Input,Output
0,Explain the remedy related to the provided bre...,Building Code of Australia 2016\n\nJ1.2 Therma...,Degas and remove AC Unit.\nInstall expanding f...
1,Communicate the solution for the given breach.,Incomplete work. \n\nHome Building Act 1989\nF...,"Attain safe access.\nClean column base,.\nPain..."
2,Explain the remedy related to the provided bre...,Daikin Installation Guide requires the followi...,Set up the work area\nMark fixing locations;\n...
3,State the breach related to the specified defect.,Waterstop does not extend to the door frame as...,Building Code of Australia 2016\nFP1.7 \nTo pr...
4,Tell the remedy for the given breachProvide th...,Daikin Installation Guide requires the followi...,Set up the work area\nMark fixing locations;\n...


In [180]:
shuffled_result.to_csv("CleanDatasetLlama3.csv")